In [78]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, sum, first, last, date_format
from pyspark.sql import Window, types

spark = SparkSession\
    .builder\
    .appName("etl")\
    .getOrCreate()

In [34]:
def cast_columns(df, columns):
    for col_name, (col_type, col_type_params) in columns.items():
        df = df.withColumn(col_name, df[col_name].cast(col_type(**col_type_params)))
    return df

def parse_json_columns(df, columns, json_columns):
    for json_col_name, schema in json_columns.items():
        df = df.withColumn(json_col_name, from_json(json_col_name, schema))\
             .select(
                *[col(col_name) for col_name in columns.keys() if not col_name == json_col_name],
                col(f'{json_col_name}.*')
        )
    return df

def remap_columns(df, columns):
    for col_name, mapping in columns.items():
        df = df.replace(to_replace=mapping, subset=[col_name])
    return df

In [35]:
csv_params = {
    'header': True,
    'inferSchema': True,
    'encoding': 'utf-8',
    'sep': ';',
}

users_df = spark.read.csv('/data/usuarios.csv', **csv_params)

users_columns = {
    'user_id': (types.StringType, {}),
    'dados_pessoais': (types.StringType, {}),
    'nivel_de_risco': (types.IntegerType, {}),
    'objetivo': (types.StringType, {}),
    'perfil_de_risco': (types.StringType, {}),
    'fez_adicional': (types.BooleanType, {}),
    'fez_resgate_parcial': (types.BooleanType, {}),
    'fez_resgate_total': (types.BooleanType, {}),
    'poupanca': (types.DecimalType, {'precision': 10, 'scale': 2}),
    'renda_fixa': (types.DecimalType, {'precision': 10, 'scale': 2}),
    'renda_variavel': (types.DecimalType, {'precision': 10, 'scale': 2}),
}

users_json_columns = {
    'dados_pessoais': types.StructType([
                          types.StructField('genero', types.StringType(), True),
                          types.StructField('estado_civil', types.StringType(), True),
                          types.StructField('idade', types.StringType(), True),
                      ]),
}


users_remap_columns = {
    'estado_civil': {
        'CASADO(A) COM BRASILEIRO(A) NATO(A)': 'CASADO(A)',
        'CASADO(A) COM BRASILEIRO(A) NATURALIZADO(A)': 'CASADO(A)',
        'UNIAO ESTAVEL': 'CASADO(A)',
    }
}

users_df.show()

+-------+--------------------+--------------+-----------------+---------------+-------------+-------------------+-----------------+---------+----------+--------------+
|user_id|      dados_pessoais|nivel_de_risco|         objetivo|perfil_de_risco|fez_adicional|fez_resgate_parcial|fez_resgate_total| poupanca|renda_fixa|renda_variavel|
+-------+--------------------+--------------+-----------------+---------------+-------------+-------------------+-----------------+---------+----------+--------------+
|      1|{"genero": "male"...|             3|     build_wealth|         medium|            1|                  0|                0| 100000.0|      null|          null|
|      2|{"genero": "femal...|             4|     build_wealth|            low|            1|                  1|                0| 190000.0|  190000.0|          null|
|      3|{"genero": "male"...|             5|     build_wealth|         medium|            1|                  0|                0|     null|    1200.0|        

In [36]:
users_df = cast_columns(users_df, users_columns)

users_df.show()

+-------+--------------------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+
|user_id|      dados_pessoais|nivel_de_risco|         objetivo|perfil_de_risco|fez_adicional|fez_resgate_parcial|fez_resgate_total|  poupanca|renda_fixa|renda_variavel|
+-------+--------------------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+
|      1|{"genero": "male"...|             3|     build_wealth|         medium|         true|              false|            false| 100000.00|      null|          null|
|      2|{"genero": "femal...|             4|     build_wealth|            low|         true|               true|            false| 190000.00| 190000.00|          null|
|      3|{"genero": "male"...|             5|     build_wealth|         medium|         true|              false|            false|      null|   1200.00|  

In [37]:
users_df = parse_json_columns(users_df, users_columns, users_json_columns)

users_df.show()

+-------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+------+--------------------+-----+
|user_id|nivel_de_risco|         objetivo|perfil_de_risco|fez_adicional|fez_resgate_parcial|fez_resgate_total|  poupanca|renda_fixa|renda_variavel|genero|        estado_civil|idade|
+-------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+------+--------------------+-----+
|      1|             3|     build_wealth|         medium|         true|              false|            false| 100000.00|      null|          null|  male|CASADO(A) COM BRA...|   40|
|      2|             4|     build_wealth|            low|         true|               true|            false| 190000.00| 190000.00|          null|female|CASADO(A) COM BRA...|   38|
|      3|             5|     build_wealth|         medium|         true|              fals

In [38]:
new_columns = {
    'idade': (types.IntegerType, {}),
}

users_df = cast_columns(users_df, new_columns)

users_df.show()

+-------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+------+--------------------+-----+
|user_id|nivel_de_risco|         objetivo|perfil_de_risco|fez_adicional|fez_resgate_parcial|fez_resgate_total|  poupanca|renda_fixa|renda_variavel|genero|        estado_civil|idade|
+-------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+------+--------------------+-----+
|      1|             3|     build_wealth|         medium|         true|              false|            false| 100000.00|      null|          null|  male|CASADO(A) COM BRA...|   40|
|      2|             4|     build_wealth|            low|         true|               true|            false| 190000.00| 190000.00|          null|female|CASADO(A) COM BRA...|   38|
|      3|             5|     build_wealth|         medium|         true|              fals

In [40]:
users_df = remap_columns(users_df, users_remap_columns)

users_df.show()

+-------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+------+-------------+-----+
|user_id|nivel_de_risco|         objetivo|perfil_de_risco|fez_adicional|fez_resgate_parcial|fez_resgate_total|  poupanca|renda_fixa|renda_variavel|genero| estado_civil|idade|
+-------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+------+-------------+-----+
|      1|             3|     build_wealth|         medium|         true|              false|            false| 100000.00|      null|          null|  male|    CASADO(A)|   40|
|      2|             4|     build_wealth|            low|         true|               true|            false| 190000.00| 190000.00|          null|female|    CASADO(A)|   38|
|      3|             5|     build_wealth|         medium|         true|              false|            false|      null|   1

In [59]:
users_df = users_df.fillna({
    'poupanca': 0,
    'renda_fixa': 0,
    'renda_variavel': 0,
})

users_df = users_df.withColumn(
        'flag_investidor_recorrente',
        users_df.fez_adicional & ~ (users_df.fez_resgate_parcial | users_df.fez_resgate_total)
)

users_df = users_df.withColumn(
        'investimentos_externos',
        sum([users_df.poupanca, users_df.renda_fixa, users_df.renda_variavel])
)

users_df.show()

+-------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+------+-------------+-----+--------------------------+----------------------+
|user_id|nivel_de_risco|         objetivo|perfil_de_risco|fez_adicional|fez_resgate_parcial|fez_resgate_total|  poupanca|renda_fixa|renda_variavel|genero| estado_civil|idade|flag_investidor_recorrente|investimentos_externos|
+-------+--------------+-----------------+---------------+-------------+-------------------+-----------------+----------+----------+--------------+------+-------------+-----+--------------------------+----------------------+
|      1|             3|     build_wealth|         medium|         true|              false|            false| 100000.00|      0.00|          0.00|  male|    CASADO(A)|   40|                      true|             100000.00|
|      2|             4|     build_wealth|            low|         true|               true|        

In [41]:
funnel_df = spark.read.csv('/data/funil.csv', **csv_params)

funnel_df.show()

+-------+-------------------+-------------+--------------+
|user_id|          timestamp|       evento|valor_simulado|
+-------+-------------------+-------------+--------------+
|      1|2017-01-06 11:40:25|     Homepage|          null|
|      1|2017-01-29 21:30:56|     Homepage|          null|
|      1|2017-01-29 21:40:04|     Homepage|          null|
|      1|2017-01-29 21:40:04|     Homepage|          null|
|      1|2017-08-01 18:40:27|     Homepage|          null|
|      1|2017-08-03 11:18:19|     Homepage|          null|
|      1|2017-08-03 14:08:16|     Homepage|          null|
|      1|2017-08-08 14:30:37|     Homepage|          null|
|      1|2017-08-15 16:47:01|Questionnaire|        120000|
|      1|2017-08-15 17:27:11|Questionnaire|        120000|
|      1|2017-08-15 17:27:58|Questionnaire|        120000|
|      1|2017-08-15 17:40:30|Questionnaire|        300000|
|      1|2017-08-15 21:15:12|     Homepage|          null|
|      1|2017-08-15 21:29:36|Questionnaire|        30000

In [75]:
funnel_columns = {
    'user_id': (types.IntegerType, {}),
    'timestamp': (types.TimestampType, {}),
    'evento': (types.StringType, {}),
    'valor_simulado': (types.DecimalType, {'precision': 10, 'scale': 2}),
}

funnel_df = cast_columns(funnel_df, funnel_columns)

funnel_df.show()

+-------+-------------------+-------------+--------------+
|user_id|          timestamp|       evento|valor_simulado|
+-------+-------------------+-------------+--------------+
|      1|2017-01-06 11:40:25|     Homepage|          null|
|      1|2017-01-29 21:30:56|     Homepage|          null|
|      1|2017-01-29 21:40:04|     Homepage|          null|
|      1|2017-01-29 21:40:04|     Homepage|          null|
|      1|2017-08-01 18:40:27|     Homepage|          null|
|      1|2017-08-03 11:18:19|     Homepage|          null|
|      1|2017-08-03 14:08:16|     Homepage|          null|
|      1|2017-08-08 14:30:37|     Homepage|          null|
|      1|2017-08-15 16:47:01|Questionnaire|     120000.00|
|      1|2017-08-15 17:27:11|Questionnaire|     120000.00|
|      1|2017-08-15 17:27:58|Questionnaire|     120000.00|
|      1|2017-08-15 17:40:30|Questionnaire|     300000.00|
|      1|2017-08-15 21:15:12|     Homepage|          null|
|      1|2017-08-15 21:29:36|Questionnaire|     300000.0

In [76]:
join = users_df.join(funnel_df, ['user_id'], how='left')

join.show()

+-------+--------------+------------+---------------+-------------+-------------------+-----------------+---------+----------+--------------+------+------------+-----+--------------------------+----------------------+-------------------+-------------+--------------+
|user_id|nivel_de_risco|    objetivo|perfil_de_risco|fez_adicional|fez_resgate_parcial|fez_resgate_total| poupanca|renda_fixa|renda_variavel|genero|estado_civil|idade|flag_investidor_recorrente|investimentos_externos|          timestamp|       evento|valor_simulado|
+-------+--------------+------------+---------------+-------------+-------------------+-----------------+---------+----------+--------------+------+------------+-----+--------------------------+----------------------+-------------------+-------------+--------------+
|      1|             3|build_wealth|         medium|         true|              false|            false|100000.00|      0.00|          0.00|  male|   CASADO(A)|   40|                      true|     

In [85]:
window = Window\
    .partitionBy(users_df.user_id)\
    .orderBy(funnel_df.timestamp)\
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

result = join.select(
        users_df.user_id, users_df.genero, users_df.estado_civil, users_df.idade, users_df.nivel_de_risco,
        users_df.objetivo, users_df.perfil_de_risco,
        date_format(first(funnel_df.timestamp, True).over(window), 'yyyy-MM').alias('homepage'),
        last(funnel_df.valor_simulado, True).over(window).alias('valor_simulado'),
        users_df.flag_investidor_recorrente, users_df.investimentos_externos
).distinct()

result.show()

+-------+------+-------------+-----+--------------+-----------------+---------------+--------+--------------+--------------------------+----------------------+
|user_id|genero| estado_civil|idade|nivel_de_risco|         objetivo|perfil_de_risco|homepage|valor_simulado|flag_investidor_recorrente|investimentos_externos|
+-------+------+-------------+-----+--------------+-----------------+---------------+--------+--------------+--------------------------+----------------------+
|     51|female|    CASADO(A)|   51|             2|     build_wealth|            low| 2017-06|      10000.00|                     false|             200000.00|
|      7|  male|  SOLTEIRO(A)|   44|             5|     build_wealth|           high| 2017-03|      20000.00|                     false|             300000.00|
|     15|female|     VIUVO(A)|   73|             3|secure_retirement|            low| 2017-11|      15000.00|                      true|            1200000.00|
|     54|  male|  SOLTEIRO(A)|   28|    

In [90]:
result.toPandas().to_csv('/data/result.csv', header=True, sep=';')